In [ ]:
import time
import numpy as np
import random

from typing import List, Tuple
from itertools import product
import pandas as pd
from functools import wraps

# Decorator to time the function execution
def time_function(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record the start time
        result = func(*args, **kwargs)  # Call the actual function
        end_time = time.time()  # Record the end time
        elapsed_time = round(end_time - start_time, 4)
        print(f"Function '{func.__name__}' executed in {elapsed_time} seconds.")
        return result  # Return the result of the original function
    return wrapper

# Function to take in a sequence of cards and return the scores for both players
def run_game(sequence:List[str], p1_seq:str, p2_seq:str) -> Tuple[int,int]: 
    if p1_seq == p2_seq:
        return 0, 0, 'T', 0, 0, 'T' # if both players guess the same, return tie
    stack = ''
    p1score_trick,p1score_cards,p2score_trick,p2score_cards = 0,0,0,0

    for card in format(sequence, '052b'):
        stack+=card
        curstack = stack[-3:]
        if curstack == p1_seq:
            p1score_trick+=1 # using "trick" method to count
            p1score_cards+=len(stack)
            stack = ''
        elif curstack == p2_seq:
            p2score_trick+=1
            p2score_cards+=len(stack)
            stack = ''

    trick_outcome = 'p1' if p1score_trick > p2score_trick else 'p2' if p2score_trick > p1score_trick else 'T'
    cards_outcome = 'p1' if p1score_cards > p2score_cards else 'p2' if p2score_cards > p1score_cards else 'T'
    
    return p1score_trick, p2score_trick, trick_outcome, p1score_cards, p2score_cards, cards_outcome # (if positive, p1 won. If negative, p2 won. If 0, tie)


# Function to simulate a number of games, ngames.
def sim(ngames: int) -> pd.DataFrame:

    # Store the deck as a string of 1s and 0s or an integer. 
    # 1s represent red cards and 0s represent black cards.
    # Test at larger scale and see if it can handle the operations. 

    patterns = [
        '111',
        '110',
        '101',
        '100',
        '011',
        '010',
        '001',
        '000'
    ]
    
    rcards = '1' * 26 
    bcards = '0' * 26
    cards = rcards+bcards # total 52 cards
    # results = np.empty((ngames * len(patterns) ** 2, 8), dtype=object)
    results = np.empty((ngames * len(patterns) ** 2, 10), dtype=object)
    
    index = 0
    for seed in range(ngames):
        random.seed(seed)

        ndeck = int(''.join(random.sample(cards, len(cards), )), 2)  # Use cards.copy() to keep the original list intact
        
        for p1guess, p2guess in product(patterns, patterns):
            result = run_game(ndeck, p1guess, p2guess)
            results[index] = [seed, ndeck, p1guess, p2guess, *result]
            index += 1
    
    return results


##########################
######## Testing #########
##########################

#if __name__ == "__main__":
    # final_old = sim(ngames = 1000)
    # final_new = optimized_sim(ngames = 1000)
  #  res = time_function(sim)(ngames=1000)  # Call the sim function with timing

  #  print(res[0])

In [ ]:
import random
import numpy as np
import sqlite3
import pandas as pd

In [ ]:
import penney_db1
from importlib import reload
reload(penney_db1);
from  penney_db1 import DB

In [ ]:
def make_database(simulation):
    db = DB()
    db.connect_db()
    db.insert_results(simulation)
    return db.get_database_file()

In [ ]:
df = pd.DataFrame(results, columns=['seed','deck','p1_seq', 'p2_seq','p1_num_tricks','p2_num_tricks','win_tricks','p1_num_cards','p2_num_cards','win_cards'])
database = make_database(df)

In [ ]:
db = DB()
db.connect_db()

In [ ]:
sql2 = '''
    WITH TricksWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_tricks = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ), 
    CardWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_cards), 0) AS p1_wins_by_cards
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_cards = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ),
    TimesPlayed AS (
        SELECT 
            p1_seq, 
            p2_seq, 
            COUNT(*) AS TimesPlayed
        FROM 
            win_results
        GROUP BY 
            p1_seq, 
            p2_seq
    )
    
    SELECT 
        t.p1_seq, 
        t.p2_seq, 
        IFNULL(t.p1_wins_by_tricks, 0) AS P1TrickWinCount,
        IFNULL(c.p1_wins_by_cards, 0) AS P1CardWinCount,
        tp.TimesPlayed, 
        IFNULL(1.0 * t.p1_wins_by_tricks / tp.TimesPlayed, 0) AS P1ProbWinTricks,
        IFNULL(1.0 * c.p1_wins_by_cards / tp.TimesPlayed, 0) AS P1ProbWinCard
    FROM 
        TricksWins t
    LEFT JOIN 
        CardWins c ON t.p1_seq = c.p1_seq AND t.p2_seq = c.p2_seq
    LEFT JOIN 
        TimesPlayed tp ON t.p1_seq = tp.p1_seq AND t.p2_seq = tp.p2_seq
    ORDER BY t.p1_seq, t.p2_seq desc;
    '''
prob_df = db.run_query(sql2)

In [ ]:
#order needs to be confirmed with the data
def create_heatmap(results):
    import seaborn as sns
    import numpy as np
    import matplotlib.pyplot as plt
    db = DB()
    db.connect_db()
    #probably need to edit/simplify/clean up this sql code to what the database given is, just using the previous code as a placeholder
    sql2 = '''
    WITH TricksWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_tricks), 0) AS p1_wins_by_tricks
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_tricks = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ), 
    CardWins AS (
        SELECT 
            all_combinations.p1_seq, 
            all_combinations.p2_seq, 
            COALESCE(COUNT(win_results.win_cards), 0) AS p1_wins_by_cards
        FROM 
            (SELECT DISTINCT p1_seq, p2_seq FROM win_results) AS all_combinations
        LEFT JOIN 
            win_results ON all_combinations.p1_seq = win_results.p1_seq 
                        AND all_combinations.p2_seq = win_results.p2_seq
                        AND win_results.win_cards = 'p1'
        GROUP BY 
            all_combinations.p1_seq, 
            all_combinations.p2_seq
    ),
    TimesPlayed AS (
        SELECT 
            p1_seq, 
            p2_seq, 
            COUNT(*) AS TimesPlayed
        FROM 
            win_results
        GROUP BY 
            p1_seq, 
            p2_seq
    )
    
    SELECT 
        t.p1_seq, 
        t.p2_seq, 
        IFNULL(t.p1_wins_by_tricks, 0) AS P1TrickWinCount,
        IFNULL(c.p1_wins_by_cards, 0) AS P1CardWinCount,
        tp.TimesPlayed, 
        IFNULL(1.0 * t.p1_wins_by_tricks / tp.TimesPlayed, 0) AS P1ProbWinTricks,
        IFNULL(1.0 * c.p1_wins_by_cards / tp.TimesPlayed, 0) AS P1ProbWinCard
    FROM 
        TricksWins t
    LEFT JOIN 
        CardWins c ON t.p1_seq = c.p1_seq AND t.p2_seq = c.p2_seq
    LEFT JOIN 
        TimesPlayed tp ON t.p1_seq = tp.p1_seq AND t.p2_seq = tp.p2_seq
    ORDER BY t.p1_seq, t.p2_seq desc;
    '''
    prob_df = db.run_query(sql2)

    prob_p1card=np.reshape(prob_df['P1ProbWinCard'].values, (8,8))
    prob_p1trick=np.reshape(prob_df['P1ProbWinTricks'].values, (8,8))

    n = prob_df['TimesPlayed'][0]
    
    viz_input={'Cards':prob_p1card, 'Tricks':prob_p1trick, 'n':n}
    var_cards = viz_input['Cards']
    var_tricks = viz_input['Tricks']
    n = viz_input['n']
    #we can make the stipulation that new data has to be entered in this format so the x an y labels don't change
    x_axis_labels = ['BBB','BBR','BRB','BRR','RBB','RBR','RRB','RRR'] #player 1 
    y_axis_labels = ['BBB','BBR','BRB','BRR','RBB','RBR','RRB','RRR'] #player 2

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

    sns.heatmap(var_cards, annot= True, annot_kws={"size": 7},linewidths=.5, fmt=".2f", cmap='flare', 
                ax=ax1, cbar=False, xticklabels=x_axis_labels, yticklabels=y_axis_labels)
    ax1.set_title('Win by Cards')
    ax1.set_xlabel('Player 1 Guesses', fontsize=7)
    ax1.set_ylabel('Player 2 Guesses', fontsize=7)

    sns.heatmap(var_tricks, annot= True, annot_kws={"size": 7},linewidths=.5, fmt=".2f", cmap='flare', 
                ax=ax2, cbar=False, xticklabels=x_axis_labels, yticklabels=y_axis_labels)
    ax2.set_title('Win by Tricks')
    ax2.set_xlabel('Player 1 Guesses', fontsize=7)
    ax2.set_ylabel('Player 2 Guesses', fontsize=7)
    
    cbar_ax = fig.add_axes([.95, 0.11, 0.02, .77])
    cb = fig.colorbar(ax1.collections[0], cax=cbar_ax)
    cb.outline.set_linewidth(.2)

    plt.subplots_adjust(wspace=0.25)
    plt.suptitle('Probability of Player 1 Winning The Penney Game (n='+str(n)+')', y=1.07, fontsize = 15)
    plt.savefig('figures/heatmap_n'+str(n)+'.png', bbox_inches = 'tight', facecolor = 'white')
    #the figures/ in the name move it to a folder I have created in the same folder I am working called figures
    plt.show()

In [ ]:
create_heatmap(results)